### **1. Importing Libraries**





In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import save_model
import pandas as pd
import numpy as np
from tensorflow.keras import Model

### **2. Loading & Processing Data**





In [14]:
def data_load():
  (X_train, y_train),(X_test, y_test) = tf.keras.datasets.mnist.load_data()
  X_train = X_train/255.0
  X_test = X_test/255.0
  X_train = X_train[..., tf.newaxis].astype('float32')
  X_test = X_test[..., tf.newaxis].astype('float32')
  return X_train, y_train, X_test, y_test
X_train, y_train, X_test, y_test =data_load()

In [15]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [16]:
## Batching and shuffling Data

def data_batch(X_train, y_train, X_test, y_test):
  train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(10000).batch(32)
  test_ds = tf.data.Dataset.from_tensor_slices(
      (X_test, y_test)).shuffle(10000).batch(32)
  return train_ds, test_ds
train_ds,test_ds = data_batch(X_train, y_train, X_test, y_test)

### **3. Creating Class with tf.keras subclass Model**

In [17]:

class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32,3, activation='relu')
    self.conv2 = Conv2D(64,3, activation='relu')
    self.flatten = Flatten()
    self.dropout = Dropout(0.25)
    self.maxpool = MaxPooling2D(2,2)
    self.d1 = Dense(1024, activation='relu')
    self.d2 = Dense(128, activation='relu')
    self.d3 = Dense(10)
  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool(x)
    x = self.conv2(x)
    x = self.maxpool(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.dropout(x)
    x = self.d2(x)
    return self.d3(x)

In [18]:
# creating instance of model
model = MyModel()

### **4. Optimizer and loss Function** 

In [19]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
Optimiser = tf.keras.optimizers.Adam()

### **5. Metrics For Loss and Acuracy**

In [20]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### **6. Function for training and Testing**

In [21]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    prediction = model(images, training=True)
    loss = loss_object(labels,prediction)
  gradient = tape.gradient(loss, model.trainable_variables)
  Optimiser.apply_gradients(zip(gradient, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, prediction)

In [22]:
@tf.function
def test_step(images, labels):
  prediction = model(images, training = False)
  t_loss = loss_object(labels, prediction)
  test_loss(t_loss)
  test_accuracy(labels, prediction)

### **7. Training Model**

In [23]:
EPOCHS = 5
min_Loss = 0
min_Accuracy = 0

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
      f'Epoch {epoch + 1}, '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result()*100}, '
      f'Test Loss: {test_loss.result()}, '
      f'Test Accuracy: {test_accuracy.result()*100}'
  )

  ### Implementing  CallBack

  if epoch==0:
    min_Loss = train_loss.result()
    min_Accuracy = train_accuracy.result()*100
  if (min_Loss>train_loss.result()):
    if (min_Accuracy <= train_accuracy.result()*100) :
      min_Loss = train_loss.result()
      min_Accuracy = ( train_accuracy.result()*100)
      #path = 'BS_'+ str(epoch + 1)+'_'+ str(min_Loss)+'_'+str(min_Accuracy) +'.h5'
      print(f"Saving Best Model {epoch+1}")
      path = f'/content/drive/MyDrive/Colab Notebooks/Model/BS_{epoch + 1}_{train_loss.result()}_{train_accuracy.result()*100}.h5'  

      model.save_weights(path)                            # Saving Model To drive
      #tf.keras.models.save_model(model, path, overwrite=True, include_optimizer=True, save_format='tf',
      #                           signatures=None, options=None, save_traces=True)
 

Epoch 1, Loss: 0.11896396428346634, Accuracy: 96.3133316040039, Test Loss: 0.04206095635890961, Test Accuracy: 98.61000061035156
Epoch 2, Loss: 0.044934533536434174, Accuracy: 98.6050033569336, Test Loss: 0.04038241133093834, Test Accuracy: 98.75
Saving Best Model 2
Epoch 3, Loss: 0.03253794461488724, Accuracy: 99.0250015258789, Test Loss: 0.02983248420059681, Test Accuracy: 99.11000061035156
Saving Best Model 3
Epoch 4, Loss: 0.0240387674421072, Accuracy: 99.24333190917969, Test Loss: 0.04142998903989792, Test Accuracy: 98.79999542236328
Saving Best Model 4
Epoch 5, Loss: 0.0192902609705925, Accuracy: 99.4183349609375, Test Loss: 0.03751206770539284, Test Accuracy: 99.04000091552734
Saving Best Model 5





*   Template From TensorFlow 2 quickstart for experts




